<a href="https://colab.research.google.com/github/3D-Lidar-FYP/Sensor_Fusion_FYP/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras==2.1.5

In [ ]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import sys
import os
import tensorflow as tf
from keras import optimizers
from keras.layers import Input, Add, concatenate
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Dropout
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import Lambda
from keras.utils import np_utils
import h5py
from matplotlib.pyplot import imshow
import glob
import math
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K

from keras.models import load_model


Using TensorFlow backend.


In [ ]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/Colab\ Notebooks

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

Files in Drive:
'Copy of Welcome To Colaboratory'    train_points.npy
'Data Processing (bdb373c6).ipynb'   Untitled0.ipynb
'Data Processing.ipynb'		     Untitled10.ipynb
 Evaluation.ipynb		     Untitled1.ipynb
 intermediate_output.npy	     Untitled2.ipynb
 my_model_weights_450.h5	     Untitled3.ipynb
 processed_image.npy		     Untitled4.ipynb
 train_classes.npy		     Untitled5.ipynb
 trainHistoryDict_history450	     Untitled6.ipynb
 Training.ipynb			     Untitled8.ipynb
 train_labels.npy


In [ ]:
points =  np.load('drive/Colab Notebooks/train_points.npy')
labels = np.load('drive/Colab Notebooks/train_labels.npy')
#labels = labels.reshape((7481,24))
labels = labels.reshape((5401,24))
classes = np.load('drive/Colab Notebooks/train_classes.npy')
classesNP= classes

In [ ]:
#intermediate_output = np.load('drive/Colab Notebooks/processed_image.npy') #I had written this
intermediate_output = np.load('drive/Colab Notebooks/intermediate_output.npy')
intermediate_output = np.squeeze(intermediate_output)
print(intermediate_output.shape)

(5402, 1000)


Model Definition:

In [ ]:

def mat_mul(A, B):
    return tf.matmul(A, B)

# number of points in each sample
num_points = 2048

# number of categories
k = 3

# define optimizer
adam = optimizers.Adam(lr=0.001, decay=0.7)

# ------------------------------------ Pointnet Architecture
# input_Transformation_net
input_points = Input(shape=(num_points, 3))
x = Convolution1D(64, 1, activation='relu',
                  input_shape=(num_points, 3))(input_points)
#x = BatchNormalization()(x)
x = Convolution1D(128, 1, activation='relu')(x)
#x = BatchNormalization()(x)
x = Convolution1D(1024, 1, activation='relu')(x)
#x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=num_points)(x)
x = Dense(512, activation='relu')(x)
#x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
#x = BatchNormalization()(x)
x = Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
input_T = Reshape((3, 3))(x)

# forward net
g = Lambda(mat_mul, arguments={'B': input_T})(input_points)
g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
#g = BatchNormalization()(g)
g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
#g = BatchNormalization()(g)

# feature transform net
f = Convolution1D(64, 1, activation='relu')(g)
#f = BatchNormalization()(f)
f = Convolution1D(128, 1, activation='relu')(f)
#f = BatchNormalization()(f)
f = Convolution1D(1024, 1, activation='relu')(f)
#f = BatchNormalization()(f)
f = MaxPooling1D(pool_size=num_points)(f)
f = Dense(512, activation='relu')(f)
#f = BatchNormalization()(f)
f = Dense(256, activation='relu')(f)
#f = BatchNormalization()(f)
f = Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
feature_T = Reshape((64, 64))(f)

# forward net
g = Lambda(mat_mul, arguments={'B': feature_T})(g)
g = Convolution1D(64, 1, activation='relu')(g)
#g = BatchNormalization()(g)
g = Convolution1D(128, 1, activation='relu')(g)
#g = BatchNormalization()(g)
g = Convolution1D(1024, 1, activation='relu')(g)
#g = BatchNormalization()(g)

# global_feature
global_feature = MaxPooling1D(pool_size=num_points)(g)
global_feature = Flatten()(global_feature)
# point_net_cls
#c = Dense(512, activation='relu')(global_feature)
#c = BatchNormalization()(c)
#c = Dropout(rate=0.7)(c)
#c = Dense(256, activation='relu')(c)
#c = BatchNormalization()(c)
#c = Dropout(rate=0.7)(c)
#c = Dense(k, activation='softmax')(c)
#prediction = Flatten()(c)
# --------------------------------------------------end of pointnet

#Fusion

resnet_activation = Input(shape=(intermediate_output.shape[1],), name='intermediate_output')
#resnet_activation = Input(shape=intermediate_output.shape, name='intermediate_output') #I had written this
f = Concatenate()([global_feature, resnet_activation])

#Definition of MLP Layer
f = Dense(512, activation='relu')(f)
f = Dense(128, activation='relu')(f)
f = Dense(128, activation='relu')(f)
boxes = Dense(labels.shape[-1])(f)
classes = Dense(classes.shape[-1],activation ='sigmoid')(f)


# print the model summary
model = Model(inputs=[input_points, resnet_activation], outputs=[boxes, classes])
print(model.summary())


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2048, 3)      0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 2048, 3)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 2048, 64)     256         lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 2048, 64)     4160        conv1d_4[0][0]                   
__________

Load Data:

In [ ]:
#skipping this block for now
index = np.load('permuted_indices.npy')

train_points = points[index[0:6750]]
dev_points = points[index[6750:7115]]
test_points = points[index[7115:]]

train_classes = classes[index[0:6750]]
dev_classes = classes[index[6750:7115]]
test_classes = classes[index[7115:]]

train_labels = labels[index[0:6750]]
dev_labels = labels[index[6750:7115]]
test_labels = labels[index[7115:]]

train_intermediate = intermediate_output[index[0:6750]]
dev_intermediate = intermediate_output[index[6750:7115]]
test_intermediate = intermediate_output[index[7115:]]

print(train_points.shape)
print(train_labels.shape)
print(train_classes.shape)
print(train_intermediate.shape)

print(dev_points.shape)
print(dev_labels.shape)
print(dev_classes.shape)
print(dev_intermediate.shape)

print(test_points.shape)
print(test_labels.shape)
print(test_classes.shape)
print(test_intermediate.shape)

In [ ]:
# I wrote this
train_points = points[0:4000]
dev_points = points[4000:5000]
test_points = points[5000:5401]

train_classes = classesNP[0:4000]
dev_classes = classesNP[4000:5000]
test_classes = classesNP[5000:5401]

train_labels = labels[0:4000]
dev_labels = labels[4000:5000]
test_labels = labels[5000:5401]

train_intermediate = intermediate_output[0:4000]
dev_intermediate = intermediate_output[4000:5000]
test_intermediate = intermediate_output[5000:5401]

print(train_points.shape)
print(train_labels.shape)
print(train_classes.shape)
print(train_intermediate.shape)

print(dev_points.shape)
print(dev_labels.shape)
print(dev_classes.shape)
print(dev_intermediate.shape)

print(test_points.shape)
print(test_labels.shape)
print(test_classes.shape)
print(test_intermediate.shape)

(4000, 2048, 3)
(4000, 24)
(4000, 3)
(4000, 1000)
(1000, 2048, 3)
(1000, 24)
(1000, 3)
(1000, 1000)
(401, 2048, 3)
(401, 24)
(401, 3)
(401, 1000)


Training:

In [ ]:
HUBER_DELTA = 0.5
def smoothL1(y_true, y_pred):
   x   = K.abs(y_true - y_pred)
   x   = K.switch(x < HUBER_DELTA, 0.5 * x ** 2, HUBER_DELTA * (x - 0.5 * HUBER_DELTA))
   return  K.sum(x)
  
  
#epoch number
epo = 450
# define optimizer
adam = optimizers.Adam(lr=0.001, decay=0.7)
# compile classification model
model.compile(optimizer='adam',
              loss=[smoothL1, 'mean_squared_error'],
              metrics=['accuracy'])

history = model.fit(x = [train_points, train_intermediate], y= [train_labels, train_classes], batch_size=32, epochs=epo, validation_data=([dev_points,dev_intermediate], [dev_labels, dev_classes]), shuffle=True, verbose=1)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 4000 samples, validate on 1000 samples
Epoch 1/450
4000/4000 [==============================] - 37s 9ms/step - loss: 2223.1518 - dense_10_loss: 2223.0425 - dense_11_loss: 0.1098 - dense_10_accuracy: 0.1262 - dense_11_accuracy: 0.8170 - val_loss: 2086.5570 - val_dense_10_loss: 2048.2686 - val_dense_11_loss: 0.0933 - val_dense_10_accuracy: 0.2830 - val_dense_11_accuracy: 0.8280
Epoch 2/450
4000/4000 [==============================] - 30s 8ms/step - loss: 2123.8620 - dense_10_loss: 2123.7642 - dense_11_loss: 0.0977 - dense_10_accuracy: 0.1190 - dense_11_accuracy: 0.8170 - val_loss: 2060.7110 - val_dense_10_loss: 2023.5381 - val_dense_11_loss: 0.0967 - val_dense_10_accuracy: 0.1920 - val_dense_11_accuracy: 0.8280
Epoch 3/450
4000/4000 [==============================] - 31s 8ms/step - loss: 2070.0148 - dense_10_loss: 2069.9165 - dense_11_loss: 0.0981 - dense_10_accuracy: 0.1447 - dense_11

In [ ]:
#model.save('/drive/Colab Notebook/current_model')
import pickle

with open('drive/Colab Notebooks/trainHistoryDict_history450', 'wb') as file_pi:
     pickle.dump(history.history, file_pi)

In [ ]:
model.save_weights('drive/Colab Notebooks/my_model_weights_450.h5')

In [ ]:
# Evaluating the model on the test data    
loss = model.evaluate([test_points, test_intermediate], [test_labels, test_classes], verbose=0)
print('Test Loss:', loss)

Test Loss: [880.5594246500448, 868.686279296875, 0.059154532849788666, 0.23940148949623108, 0.8977556228637695]


In [ ]:
#Evaluating model of Dev Set
loss = model.evaluate([dev_points, dev_intermediate], [dev_labels, dev_classes], verbose=0)
print('Dev Loss:', loss)

Dev Loss: [836.7958598632813, 820.1350708007812, 0.06046322360634804, 0.23800000548362732, 0.8939999938011169]
